In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from collections import defaultdict

import sqlite3
import sys
import time
import math
import tqdm

from glicko2 import Player

## Loading SQLite Database into Pandas DataFrames

The following code connects to an SQLite database (`melee_player_database.db`) and converts each table within the database into a pandas DataFrame. The DataFrames will be stored in a dictionary, where each key corresponds to the table name with `_df` appended, and the values are the respective DataFrames.

### Steps:

1. **Database Connection**: We use the `sqlite3` library to connect to the SQLite database file.
2. **Retrieve Table Names**: A query retrieves all the table names in the database.
3. **Convert Tables to DataFrames**: For each table:
   - The table is loaded into a pandas DataFrame using `pd.read_sql()`.
   - We check each column to see if any data is JSON-formatted (lists or dictionaries). If so, we convert these columns from strings into their corresponding Python objects using `json.loads()`.
4. **Store DataFrames**: The DataFrames are stored in a dictionary, where the key is the table name with a `_df` suffix, and the value is the DataFrame.
5. **Database Connection Closed**: Once all tables are loaded into DataFrames, the database connection is closed.

### Example:
If the database contains a table named `players`, the corresponding DataFrame will be stored in the dictionary with the key `players_df`, and can be accessed as:

```python
players_df = dfs['players_df']


In [35]:
# Function to get the table names
def get_table_names(conn):
    query = "SELECT name FROM sqlite_master WHERE type='table';"
    return pd.read_sql(query, conn)['name'].tolist()

# Function to load tables into DataFrames
def load_tables_to_dfs(conn):
    table_names = get_table_names(conn)
    dataframes = {}
    
    for table in table_names:
        # Load table into a DataFrame
        df = pd.read_sql(f"SELECT * FROM {table}", conn)
        
        # Detect and convert JSON formatted columns (if any)
        for col in df.columns:
            # Check if any entry in the column is a valid JSON (list or dictionary)
            if df[col].apply(lambda x: isinstance(x, str)).all():
                try:
                    # Try parsing the column as JSON
                    df[col] = df[col].apply(lambda x: json.loads(x) if pd.notnull(x) else x)
                except (json.JSONDecodeError, TypeError):
                    # If it fails, skip the column
                    pass
        
        # Store the DataFrame with table name + '_df'
        dataframes[f"{table}_df"] = df
        
    return dataframes

# Check if the flag variable exists in the global scope so that this code does not run twice
if 'cell_has_run' not in globals():
    path = "../data/melee_player_database.db"
    
    # Connect to the database
    conn = sqlite3.connect(path)

    # Convert each table into a DataFrame
    dfs = load_tables_to_dfs(conn)

    # Close the connection
    conn.close()

    # Now, you have a dictionary 'dfs' where each key is the table name with '_df' suffix and value is the corresponding DataFrame.
    # For example, to access the DataFrame for a table called 'players':
    # players_df = dfs['players_df']

    dfs['tournament_info_df']['start'] = pd.to_datetime(dfs['tournament_info_df']['start'], unit='s')
    dfs['tournament_info_df']['end'] = pd.to_datetime(dfs['tournament_info_df']['end'], unit='s')

    
    # Set the flag to indicate that the cell has been run
    cell_has_run = True

### Here we adjust the data types of the dataframes so that they are the correct type. (This will be updated as needed.)

In [36]:
dfs['sets_df']['best_of'] = dfs['sets_df']['best_of'].fillna(0).astype(int) 

### Here we make dataframes that we will use and print the head.

The integers in 'characters' count the number of games the player has played that character. (We verify this for Zain below.)

In [37]:
players_df = dfs['players_df']
players_df.head()

,game,player_id,tag,all_tags,prefixes,social,country,state,region,c_country,c_state,c_region,placings,characters,alias
0,melee,Rishi,Rishi,[Rishi],[],{'twitter': []},,,,None,None,None,[{'key': 'mdva-invitational-2017-(challonge-mi...,,None
1,melee,15634,lloD,"[lloD, VGz | lloD, Llod]",[],{'twitter': ['lloD74']},United States,VA,,US,CA,Laurel,[{'key': 'mdva-invitational-2017-(challonge-mi...,"{'melee/peach': 1089, 'melee/falco': 1, 'melee...",None
2,melee,6126,Zain,"[Zain, DontTestMe]",[PG],{'twitter': ['PG_Zain']},United States,VA,,US,CA,Los Angeles,[{'key': 'mdva-invitational-2017-(challonge-mi...,"{'melee/marth': 1065, 'melee/pichu': 1, 'melee...",DontTestMe
3,melee,Chu,Chu,[Chu],[],{'twitter': []},,,,None,None,None,[{'key': 'mdva-invitational-2017-(challonge-mi...,,None
4,melee,5620,Junebug,"[Junebug, LS | VGz Junebug]",[],{'twitter': ['arJunebug']},United States,VA,,US,VA,Richmond,[{'key': 'mdva-invitational-2017-(challonge-mi...,"{'melee/sheik': 46, 'melee/falco': 4, 'melee/g...",None


In [38]:
ranking_df = dfs['ranking_df']
ranking_df.head()

,game,ranking_name,priority,region,seasons,tournaments,icon
0,melee,SSBMRank,0,world,"[2015, 2016, 2017, 2018, 2019]",[],miom


In [39]:
ranking_seasons_df = dfs['ranking_seasons_df']
ranking_seasons_df.head()

,game,ranking_name,season,start,end,total,by_id,by_placing,final,name
0,melee,SSBMRank,2015,1420070400,1451606399,100,"{'6189': 1, '1004': 2, '4465': 3, '1000': 4, '...","{'1': '6189', '2': '1004', '3': '4465', '4': '...",0,None
1,melee,SSBMRank,2016,1451606400,1483228799,100,"{'6189': 1, '1004': 2, '1000': 3, '1003': 4, '...","{'1': '6189', '2': '1004', '3': '1000', '4': '...",0,None
2,melee,SSBMRank,2017,1483228800,1514764799,100,"{'1004': 1, '6189': 2, '1000': 3, '1003': 4, '...","{'1': '1004', '2': '6189', '3': '1000', '4': '...",0,None
3,melee,SSBMRank,2018,1514793600,1546329600,100,"{'1004': 1, '6189': 2, '4465': 3, '15990': 4, ...","{'1': '1004', '2': '6189', '3': '4465', '4': '...",0,None
4,melee,SSBMRank,2019,1546329600,1577836800,100,"{'1004': 1, '4465': 2, '1000': 3, '16342': 4, ...","{'1': '1004', '2': '4465', '3': '1000', '4': '...",0,None


In [40]:
sets_df = dfs['sets_df']
print(f'{sets_df[sets_df['game_data'].apply(lambda x: len(x) > 0)].shape[0] / sets_df.shape[0]:0.01%} percent of sets have some game data')
sets_df.head()

32.9% percent of sets have some game data


,key,game,tournament_key,winner_id,p1_id,p2_id,p1_score,p2_score,location_names,bracket_name,bracket_order,set_order,best_of,game_data
0,104675843,melee,mdva-invitational-2017-(challonge-mirror),5620,5620,Chillin,3,1,"[R1, Round 1, Round 1]",,1,A,5,[]
1,104675844,melee,mdva-invitational-2017-(challonge-mirror),Aglet,15634,Aglet,2,3,"[R1, Round 1, Round 1]",,1,B,5,[]
2,104675845,melee,mdva-invitational-2017-(challonge-mirror),6126,6126,1097,3,0,"[R1, Round 1, Round 1]",,1,C,5,[]
3,104675846,melee,mdva-invitational-2017-(challonge-mirror),1069,Chu,1069,0,3,"[R1, Round 1, Round 1]",,1,D,5,[]
4,104675847,melee,mdva-invitational-2017-(challonge-mirror),Rishi,Jerry,Rishi,1,3,"[R1, Round 1, Round 1]",,1,E,5,[]


In [41]:
tournament_info_df = dfs['tournament_info_df']
tournament_info_df.head()

,game,key,cleaned_name,source,tournament_name,tournament_event,season,rank,start,end,country,state,city,entrants,placings,losses,bracket_types,online,lat,lng
0,melee,mdva-invitational-2017-(challonge-mirror),MDVA Invitational 2017 (Challonge Mirror),challonge,https://challonge.com/mdva_invitational_2017,,17,,2017-11-26 08:05:11,2017-11-26 08:48:09,US,VA,Fall's Church,10,"[[Rishi, 1], [15634, 3], [6126, 4], [Chu, 8], ...",{},b'{}',0,NaN,NaN
1,melee,s@sh7,S@SH7,challonge,https://challonge.com/sash7,,17,,2017-06-13 10:27:01,2017-06-13 10:27:01,US,MI,Ann Arbor,92,[],{},b'{}',0,NaN,NaN
2,melee,slippi-champions-league-week-1__melee-singles,Slippi Champions League Week 1,pgstats,slippi-champions-league-week-1,melee-singles,20,,2020-10-11 14:00:00,2020-10-11 14:00:00,,,,20,"[[1000, 1], [6126, 2], [4107, 3], [19554, 3], ...",{},b'{}',1,0.0,0.0
3,melee,slippi-champions-league-week-2__melee-singles,Slippi Champions League Week 2,pgstats,slippi-champions-league-week-2,melee-singles,20,,2020-10-18 14:00:00,2020-10-18 14:00:00,,,,20,"[[6126, 1], [4107, 2], [1000, 3], [19554, 3], ...",{},b'{}',1,0.0,0.0
4,melee,slippi-champions-league-week-3__melee-singles,Slippi Champions League Week 3,pgstats,slippi-champions-league-week-3,melee-singles,20,,2020-10-25 14:00:00,2020-10-25 14:00:00,,,,20,"[[6126, 1], [3359, 2], [19554, 3], [4107, 3], ...",{},b'{}',1,0.0,0.0


In [42]:
total_character_counts = {}

# Iterate over the 'characters' column
for idx, row in players_df.iterrows():
    char_dict = row['characters']
    if isinstance(char_dict, dict):
        for character, count in char_dict.items():
            if character in total_character_counts:
                total_character_counts[character] += count
            else:
                total_character_counts[character] = count
    else:
        # Handle cases where 'characters' is empty or NaN
        continue

# Now total_character_counts contains the total counts per character

# Sort the characters by total counts in decreasing order
sorted_characters = sorted(total_character_counts.items(), key=lambda x: x[1], reverse=True)
sorted_character_keys = [character[0] for character in sorted_characters]

In [43]:
sorted_tournament_info_df = tournament_info_df.sort_values('end').reset_index(drop=True)
first_date = sorted_tournament_info_df.iloc[0]['start']


Create a dataframe with dictionaries to store the data.

In [44]:
from copy import deepcopy

# Create the 2D dictionary
character_dict = {
    char1: {char2: None for char2 in sorted_character_keys} for char1 in sorted_character_keys
}

# Initialize dictionaries for dates, ratings, and RD histories
dates_dict = deepcopy(character_dict)
rating_history_dict = deepcopy(character_dict)
rd_history_dict = deepcopy(character_dict)
glicko2_dict = deepcopy(character_dict)

inital_player = Player()
inital_date = [first_date] # the initial value that goes in all the positions of dates_dict
rating_history = [inital_player.getRating()] # the initial value that goes in all the positions of rating_history_dict
rd_history = [inital_player.getRd()] # the initial value that goes in all the positions of rd_history_dict

# Fill the dictionaries with initial values
for char1 in sorted_character_keys:
    for char2 in sorted_character_keys:
        dates_dict[char1][char2] = deepcopy(inital_date)
        rating_history_dict[char1][char2] = deepcopy(rating_history)
        rd_history_dict[char1][char2] = deepcopy(rd_history)
        rd_history_dict[char1][char2] = Player()

player_ratings_matchup_df = pd.DataFrame(columns=[
     'player_id',
     'dates_dict',
     'rating_history_dict', 
     'rd_history_dict',
     'game_counts_dict',
     'glicko2_dict'
])

player_ratings_matchup_df['player_id'] = players_df['player_id']
player_ratings_matchup_df['dates_dict'] = [dates_dict.copy for _ in range(player_ratings_matchup_df.shape[0])]
player_ratings_matchup_df['rating_history_dict'] = [rating_history_dict.copy for _ in range(player_ratings_matchup_df.shape[0])]
player_ratings_matchup_df['rd_history_dict'] = [rd_history_dict.copy for _ in range(player_ratings_matchup_df.shape[0])]
player_ratings_matchup_df['glicko2_dict'] = [Player() for _ in range(player_ratings_matchup_df.shape[0])]



In [56]:
game_data_sets_df= sets_df[sets_df['game_data'].apply(lambda x: len(x) > 0)]
seen = set()
tournament_keys = [key for key in game_data_sets_df['tournament_key'] if not (key in seen or seen.add(key))]
# Filter tournament_info_df to include only matching tournament keys
game_data_tournaments_df = tournament_info_df[
    tournament_info_df['key'].isin(tournament_keys)
].sort_values('end').reset_index(drop=True)


In [45]:
def process_tournament(tournament_key, player_ratings_df, tournament_info_df, sets_df):
    """
    Process a tournament to update player ratings.
    """
    # Get the sets for this tournament
    tournament_sets_df = sets_df[sets_df['tournament_key'] == tournament_key]
    
    # Extract the unique player IDs from the sets
    tournament_players = pd.unique(tournament_sets_df[['p1_id', 'p2_id']].values.ravel())
    
    # Add any new players to the player_ratings_df
    new_players = [player for player in tournament_players if player not in player_ratings_df.index]
    if new_players:
        new_player_df = pd.DataFrame({
            'dates': [[] for _ in new_players],
            'rating_history': [[] for _ in new_players],
            'rd_history': [[] for _ in new_players],
            'glicko2': [Player() for _ in new_players]
        }, index=new_players)
        player_ratings_df = pd.concat([player_ratings_df, new_player_df], ignore_index=False)
    
    # Ensure the index name is set to 'player_id'
    player_ratings_df.index.name = 'player_id'
    
    # Create a mapping from player_id to their Glicko2 player object
    player_map = player_ratings_df['glicko2'].to_dict()
    
    # Create a snapshot of ratings before the tournament
    ratings_snapshot = player_ratings_df[['glicko2']].copy()
    ratings_snapshot['rating'] = ratings_snapshot['glicko2'].apply(lambda x: x.getRating())
    ratings_snapshot['rd'] = ratings_snapshot['glicko2'].apply(lambda x: x.getRd())
    # Add 'opponent_id' column from the index
    ratings_snapshot.reset_index(inplace=True)
    ratings_snapshot.rename(columns={'player_id': 'opponent_id'}, inplace=True)
    ratings_snapshot = ratings_snapshot[['opponent_id', 'rating', 'rd']]
    
    # Prepare player matches DataFrame
    df_p1 = tournament_sets_df[['p1_id', 'p2_id', 'winner_id']].copy()
    df_p1.rename(columns={'p1_id': 'player_id', 'p2_id': 'opponent_id'}, inplace=True)
    df_p1['outcome'] = (df_p1['winner_id'] == df_p1['player_id']).astype(int)
    df_p1 = df_p1[['player_id', 'opponent_id', 'outcome']]
    
    df_p2 = tournament_sets_df[['p2_id', 'p1_id', 'winner_id']].copy()
    df_p2.rename(columns={'p2_id': 'player_id', 'p1_id': 'opponent_id'}, inplace=True)
    df_p2['outcome'] = (df_p2['winner_id'] == df_p2['player_id']).astype(int)
    df_p2 = df_p2[['player_id', 'opponent_id', 'outcome']]
    
    player_matches = pd.concat([df_p1, df_p2], ignore_index=True)
    
    # Merge to get opponent ratings and RDs
    player_matches = player_matches.merge(
        ratings_snapshot,
        on='opponent_id', how='left'
    )
    player_matches.rename(columns={'rating': 'opponent_rating', 'rd': 'opponent_rd'}, inplace=True)
    
    # Group by player_id to aggregate opponent data
    grouped = player_matches.groupby('player_id').agg({
        'opponent_rating': list,
        'opponent_rd': list,
        'outcome': list
    }).reset_index()
    
    # Get the tournament end date
    end_date = tournament_info_df.loc[tournament_info_df['key'] == tournament_key, 'end'].values[0]
    end_date = pd.to_datetime(end_date)
    
    # Process each player
    for idx, row in grouped.iterrows():
        player_id = row['player_id']
        rating_list = row['opponent_rating']
        rd_list = row['opponent_rd']
        outcome_list = row['outcome']
        
        # Update Glicko rating for the player
        player_glicko = player_map[player_id]
        if rating_list:  # Ensure the player has matches to process
            player_glicko.update_player(rating_list, rd_list, outcome_list)
        
        # Update the player's history
        player_ratings_df.at[player_id, 'dates'].append(end_date)
        player_ratings_df.at[player_id, 'rating_history'].append(player_glicko.getRating())
        player_ratings_df.at[player_id, 'rd_history'].append(player_glicko.getRd())
    
    return player_ratings_df  # Return the updated DataFrame

,player_id,dates_dict,rating_history_dict,rd_history_dict,glicko2_dict
0,Rishi,<built-in method copy of dict object at 0x0000...,<built-in method copy of dict object at 0x0000...,<built-in method copy of dict object at 0x0000...,<glicko2.glicko2.Player object at 0x0000020AAB...


In [ ]:
# Loop over tournaments
for idx, tournament in tqdm.tqdm(game_data_tournaments_df.iterrows(), total=sorted_tournament_info_df.shape[0]):
    player_ratings_df = process_tournament(tournament['key'], player_ratings_matchup_df, game_data_tournaments_df, game_data_sets_df)